# Import libraries

In [2]:
import tweepy
import json

# Read Twitter dev keys

In [3]:
keys_filename = "../keys.json"
consumer_key, consumer_secret, access_token, access_secret = "", "", "", ""
with open(keys_filename) as api_credentials:
    api_twitter = json.loads(api_credentials.read())
    consumer_key = api_twitter['Twitter']["consumer_key"]
    consumer_secret = api_twitter['Twitter']["consumer_secret"]
    access_token = api_twitter['Twitter']["access_token"]
    access_secret = api_twitter['Twitter']["access_secret"]

# API Setup

In [8]:
#Login
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
#Create API
api = tweepy.API(auth)

# Experimenting with tweets download

In [11]:
tweets = tweepy.Cursor(api.search,  
              q = "Assetto Corsa",
              since = "2019-06-08", 
              until = "2019-06-09").items()

## Let's have a look to the download tweets

In [12]:
for tweet in tweets:
    print(tweet)

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 23:36:45 +0000 2019', 'id': 1137503762275282945, 'id_str': '1137503762275282945', 'text': 'Joder con Assetto Corsa Competizione https://t.co/Ua3qu8S7uR', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Ua3qu8S7uR', 'expanded_url': 'https://twitter.com/teommotorsport/status/1137373229335699456', 'display_url': 'twitter.com/teommotorsport…', 'indices': [37, 60]}]}, 'metadata': {'iso_language_code': 'it', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1035799026, 'id_str': '1035799026', 'name': 'Aitxormen', 'screen_name': 'Aitxormen', 'location': '', 'description': 'Serendipiando.', 'url': None, 'entiti

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 19:07:57 +0000 2019', 'id': 1137436118004174850, 'id_str': '1137436118004174850', 'text': '@GoProLopez @iRacing is that assetto corsa in the background? :P', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'GoProLopez', 'name': 'Pablo López 🐙', 'id': 68934593, 'id_str': '68934593', 'indices': [0, 11]}, {'screen_name': 'iRacing', 'name': 'iRacing.com', 'id': 16240253, 'id_str': '16240253', 'indices': [12, 20]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': 1137435208423280640, 'in_reply_to_status_id_str': '1137435208423280640', 'in_reply_to_user_id': 68934593, 'in_reply_to_user_id_str': '68934593', 'in_reply_to_screen_name': 'GoProLopez', 'user': {'id': 97549464, 'id_str': '97549464', 'name': 'Jardier', 'screen_name':

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 16:09:20 +0000 2019', 'id': 1137391166998163456, 'id_str': '1137391166998163456', 'text': 'Assetto Corsa ese juego en el cual @MeeTryy aún llevando volante es Jolimbo con los Ferraris.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'MeeTryy', 'name': 'Meetry', 'id': 1020083626061549569, 'id_str': '1020083626061549569', 'indices': [35, 43]}], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 888026248777256960, 'id_str': '888026248777256960', 'name': 'adrián✞', 'screen_name': 'SkatedZ', 'location': 'C A R T A G E N A ', 'description': '16. CoD | @KhaIIx 

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 12:23:33 +0000 2019', 'id': 1137334346002575360, 'id_str': '1137334346002575360', 'text': 'アーリーアクセスで早期購入してから\n全然プレイしてなかったAssetto Corsa Competizione\nがいつの間にか車種もコースも増えてて進化してた件\nレースゲームの進化すごいなぁ（遠い目） https://t.co/2pvWTRiseJ', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1137333066660061184, 'id_str': '1137333066660061184', 'indices': [101, 124], 'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1137333066660061184/pu/img/dwew6aZrvnUyWi0Q.jpg', 'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1137333066660061184/pu/img/dwew6aZrvnUyWi0Q.jpg', 'url': 'https://t.co/2pvWTRiseJ', 'display_url': 'pic.twitter.com/2pvWTRiseJ', 'expanded_url': 'https://twitter.com/KDdollMoko/status/1137334346002575360/video/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 1200, 'h': 675, 'resize': 'fit'}

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 08:20:17 +0000 2019', 'id': 1137273125903749121, 'id_str': '1137273125903749121', 'text': 'Gaming: Assetto Corsa Competizione\xa0Review https://t.co/vQXDndyYY2 https://t.co/K0QttPady1', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/vQXDndyYY2', 'expanded_url': 'https://euro-journal.press/en/gaming-assetto-corsa-competizione-review/', 'display_url': 'euro-journal.press/en/gaming-asse…', 'indices': [42, 65]}], 'media': [{'id': 1137273124439961601, 'id_str': '1137273124439961601', 'indices': [66, 89], 'media_url': 'http://pbs.twimg.com/media/D8hn4ZLXsAElZaD.jpg', 'media_url_https': 'https://pbs.twimg.com/media/D8hn4ZLXsAElZaD.jpg', 'url': 'https://t.co/K0QttPady1', 'display_url': 'pic.twitter.com/K0QttPady1', 'expanded_url': 'https://twitter.com/eurojournalENG/status/1137273125903749121/photo/1', 'type': 'photo', 'sizes': {'medium': {'w

Status(_api=<tweepy.api.API object at 0x10ba21a90>, _json={'created_at': 'Sat Jun 08 03:33:43 +0000 2019', 'id': 1137201010223435781, 'id_str': '1137201010223435781', 'text': '放送開始：【Assetto Corsa】ハンコンで練習 204日目 https://t.co/tdm1aodI0z #lv320473327', 'truncated': False, 'entities': {'hashtags': [{'text': 'lv320473327', 'indices': [58, 70]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/tdm1aodI0z', 'expanded_url': 'http://live.nicovideo.jp/watch/lv320473327', 'display_url': 'live.nicovideo.jp/watch/lv320473…', 'indices': [34, 57]}]}, 'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'}, 'source': '<a href="http://com.nicovideo.jp/community/co278640" rel="nofollow">あんこついったーfor やります！アンコちゃん</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2937072182, 'id_str': '2937072182', 'name': '4AG86', 'screen_name': '4ag8686', 'location'

In [14]:
tweets.num_tweets

88

Things to do from now on:
- Create a function that, given a query, a start date and an end date returne the tweets
- Understand how to filter data: we are not interested in all the infos (I think that language + tweet + account_name + follower_count + language may be enough)
- Trasform everything in a Python script so that it will be easier to launch it on the VM

In [ ]:
for tweet in tweets:
    filtered_tweet = {
        "text": tweet.text,
        "language" : tweet.lang,
        "date"     : tweet.created_at,
        "username" : tweet.user.name,
        "user_followers": tweet.user.followers_count,
        "user_location": tweet.user.location,
        "retweets" : tweet.retweet_count,
        "likes"    : tweet.favorite_count,
    }
    print(filtered_tweet)

# Setup mongo

In [8]:
from pymongo import MongoClient
ip_mongo = "localhost"
mongo_port = 27017

client = MongoClient(ip_mongo, mongo_port)
db = client.twitter
tweet_games = db.game_tweets

In [ ]:
def collect_and_store_tweets(api, query, since_date, until_date, mongo_collection):
    tweets = tweepy.Cursor(api.search,  
              q = query,
              since = since_date, 
              until = until_date).items()
    for tweet in tweets:
        filtered_tweet = {
            "text": tweet.text,
            "language" : tweet.lang,
            "date"     : tweet.created_at,
            "username" : tweet.user.name,
            "user_followers": tweet.user.followers_count,
            "user_location": tweet.user.location,
            "retweets" : tweet.retweet_count,
            "likes"    : tweet.favorite_count,
        }
        mongo_collection.insert_one(filtered_tweet)
        

I have improved the code contained in this section and made it a script

# Testing library contained in scripts

In [15]:
import sys
sys.path.insert(0, '../scripts/') #If this doesn't work try removing the double dots or use an absolute path.

In [16]:
from collect_and_store_tweets import download_and_store_tweets_on_default_mongodb_instance

In [17]:
list_of_queries = ['Assetto Corsa', "Mario Kart 8", "Dark Souls"]

In [18]:
keys_filename = "../keys.json"
ip_mongo="local"
mongo_port=27017
download_and_store_tweets_on_default_mongodb_instance(keys_filename=keys_filename,
                                               list_of_queries=list_of_queries,
                                               since_date="2019-06-08",
                                               until_date="2019-06-09",
                                               ip_mongo=ip_mongo,
                                               mongo_port=mongo_port)

KeyboardInterrupt: 

In [19]:
query4res = db.game_tweets.find(
        {"query": "Mario Kart 8"}
    ).sort("user.followers_count", -1)
[x for x in query4res]

NameError: name 'db' is not defined